---
# Uber-Project

---

In [50]:
import datetime as dt
import pandas_profiling
import pandas as pd
import numpy as np
import seaborn as sas
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs ,init_notebook_mode, plot,iplot
init_notebook_mode(connected=True)
%matplotlib inline
sas.set()

In [51]:
American = pd.read_csv('H:/PYTHON/python for data science/s11-project/uber/other-American_B01362.csv',usecols=['DATE','TIME','PICK UP ADDRESS'])


---
## ASK
---

__1-How many people ask for a trip AM or PM ?__

__2-How many unique address we have ?__

__3-Is there a particular time people ask for trips?__

__4-What is the most day people asked trips in it__

__5-What is the number of trips for each day?__

__6-What is the number of trips for each month?__

__7-What is the number of trips for each week?__

__8-What are the most regions asked for trips?__

__9-Is there regions asked for trips only AM or  PM?__

__10-What is the most period people ask for trips in it?__

__11-What is the total number of trips?__

__12-What is the average of number trips by day, month, week, hour and minute ?__

__13-What is the top 10 address that has asked for trips and the lowest 10?__

__14-What is the total of number trips by hour and minute ?__

__15-How many region we have?__

In [52]:
American


,DATE,TIME,PICK UP ADDRESS
0,7/1/2014,12:00:00 AM,"874 E 139th St Mott Haven, BX"
1,7/1/2014,12:01:00 AM,"628 E 141st St Mott Haven, BX"
2,7/1/2014,12:01:00 AM,"601 E 156th St South Bronx, BX"
3,7/1/2014,12:01:00 AM,"708 E 138th St Mott Haven, BX"
4,7/1/2014,12:02:00 AM,"700 E 140th St Mott Haven, BX"
...,...,...,...
91707,9/30/2014,11:47:00 PM,"593 Oak Ter Mott Haven, BX"
91708,9/30/2014,11:48:00 PM,"645 Westchester Ave South Bronx, BX"
91709,9/30/2014,11:51:00 PM,"625 Tinton Ave South Bronx, BX"
91710,9/30/2014,11:54:00 PM,"350 Saint Anns Ave Mott Haven, BX"


---
## Data Cleaning and Preprocessing
---

In [53]:
American.columns = ['date','time','pick_up_address']

In [54]:
American['AM_PM'] = American.time.str.split(" ").str[1]
American['time'] = American.time.str.split(" ").str[0]

In [55]:
American=American.astype({'pick_up_address':'string','AM_PM':'string'})

In [56]:
region_with_out_st = American[~American.pick_up_address.str.contains(r'\s+St+\s')]

region_with_out_st.pick_up_address = region_with_out_st.pick_up_address.str.split(r'[\d]').str[-1].str.replace(', BX','')


C:\Users\bhr\AppData\Local\Temp\ipykernel_5284\2572964650.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [57]:
region_with_st = American[American.pick_up_address.str.contains(r'\s+St+\s')]

region_with_st.pick_up_address = region_with_st.pick_up_address.str.split(r'\d+\s+(\w?)+(\s?)+([0-9a-z]+)?\s').str[-1].str.replace(', BX','').str.replace('St ','')


C:\Users\bhr\AppData\Local\Temp\ipykernel_5284\2763703507.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [58]:
American['region']= pd.concat([region_with_out_st.pick_up_address,region_with_st.pick_up_address])
American.region = American.region.astype('string')

In [59]:
American['complete_date'] = pd.to_datetime(American['date'] + ' ' + American['time'])
American['date'] = pd.to_datetime(American['date'])

In [60]:
American['days'] = American.complete_date.dt.day_name()
American['months'] = American.complete_date.dt.month_name()
American['weeks'] = American.complete_date.dt.weekday
American['hour'] = American.complete_date.dt.hour
American['minute'] = American.complete_date.dt.minute

In [107]:
American.minute = American.minute + 1

In [110]:
American.minute.nunique()

60

In [61]:
American['AM_PM'] = American['AM_PM'].astype('object')

In [62]:
condtion = [
    (American['hour'] <= 12) & (American['hour'] <= 5) & (American['AM_PM'] == 'AM') ,
    (American['hour'] >= 6) & (American['hour'] <= 12) & (American['AM_PM'] == 'AM') ,
    (American['hour'] <= 12) & (American['hour'] <= 3) & (American['AM_PM'] == 'PM') ,
    (American['hour'] >= 4) &(American['hour'] <= 7 ) & (American['AM_PM'] == 'PM')  ,
    (American['hour'] >= 8) & (American['hour'] <= 12)& (American['AM_PM'] == 'PM') ]

choices = ['Dawn', 'Morning','After Noon','Evening','Night']    

American['time_period']= np.select(condtion,choices)

In [63]:
American[(American['date'] >= '2014-08-01') & (American['date'] <= '2014-08-31')]

,date,time,pick_up_address,AM_PM,region,complete_date,days,months,weeks,hour,minute,time_period
30348,2014-08-01,12:01:00,"530 E 137th St Mott Haven, BX",AM,Mott Haven,2014-08-01 12:01:00,Friday,August,4,12,1,Morning
30349,2014-08-01,12:02:00,"458 E 144th St Mott Haven, BX",AM,Mott Haven,2014-08-01 12:02:00,Friday,August,4,12,2,Morning
30350,2014-08-01,12:03:00,"730 E 166th St Morrisania, BX",AM,Morrisania,2014-08-01 12:03:00,Friday,August,4,12,3,Morning
30351,2014-08-01,12:03:00,"730 E 166th St Morrisania, BX",AM,Morrisania,2014-08-01 12:03:00,Friday,August,4,12,3,Morning
30352,2014-08-01,12:05:00,"281 E 153rd St Cortlandt, BX",AM,Cortlandt,2014-08-01 12:05:00,Friday,August,4,12,5,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...
61826,2014-08-31,11:56:00,"700 Brook Ave South Bronx, BX",PM,Brook Ave South Bronx,2014-08-31 11:56:00,Sunday,August,6,11,56,Night
61827,2014-08-31,11:57:00,"681 E 136th St Mott Haven, BX",PM,Mott Haven,2014-08-31 11:57:00,Sunday,August,6,11,57,Night
61828,2014-08-31,11:57:00,"346 Powers Ave Mott Haven, BX",PM,Powers Ave Mott Haven,2014-08-31 11:57:00,Sunday,August,6,11,57,Night
61829,2014-08-31,11:58:00,"1186 Union Ave Foxhurst, BX",PM,Union Ave Foxhurst,2014-08-31 11:58:00,Sunday,August,6,11,58,Night


In [64]:

American['Week_name'] = np.select(
[(American.date >= '2014-07-01') & (American.date <= '2014-07-07'),
(American.date >= '2014-09-01') & (American.date <= '2014-08-07'),
(American.date >= '2014-09-01') & (American.date <= '2014-09-07'),
(American.date >= '2014-07-08') & (American.date <= '2014-07-14'),
(American.date >= '2014-08-08') & (American.date <= '2014-08-14'),
(American.date >= '2014-09-08') & (American.date <= '2014-09-14'),
(American.date >= '2014-07-15') & (American.date <= '2014-07-21'),
(American.date >= '2014-08-15') & (American.date <= '2014-08-21'),
(American.date >= '2014-09-15') & (American.date <= '2014-09-21'),
(American.date >= '2014-07-22') & (American.date <= '2014-07-27'),
(American.date >= '2014-08-22') & (American.date <= '2014-08-27'),
(American.date >= '2014-09-22') & (American.date <= '2014-09-27'),],
['First Week','First Week','First Week','Second Week','Second Week','Second Week','Third Week','Third Week','Third Week','Fourth Week','Fourth Week','Fourth Week'] , default='Fifth Week')

In [65]:
American.Week_name.value_counts()

Second Week    21224
Third Week     20971
Fifth Week     17766
Fourth Week    17344
First Week     14407
Name: Week_name, dtype: int64

---
## Analysis and Visualization
---

__1-How many people ask for a trip AM or PM ?__

In [66]:
q_1 = pd.DataFrame(American.AM_PM.value_counts()).reset_index().rename(columns={'index':'AM_PM','AM_PM':'number_of_trips'})
q_1

,AM_PM,number_of_trips
0,PM,49377
1,AM,42335


In [67]:
fig = px.pie(q_1 ,
values='number_of_trips',
names='AM_PM' ,
title='Number of trips by AM of PM.',
template='none',
hover_name='number_of_trips')
fig.update_traces(hoverinfo = 'label+percent' ,
                  textfont_size = 20 ,
                  textinfo = 'label+percent' ,
                  pull=[0.1 , 0 , 0.2 , 0 , 0 , 0] )
                 

__2-How many unique address we have ?__

In [68]:
q_2 = pd.Series(American.pick_up_address.nunique())
q_2 = pd.DataFrame({'number_of_unique':q_2})
q_2

,number_of_unique
0,10608


__3-Is there a particular time people ask for trips?__

In [69]:
q_3 = American.groupby('time').agg(number_of_trips = ('time','count')).reset_index().sort_values(by='number_of_trips' , ascending=False)
q_3.head(10)

,time,number_of_trips
660,9:00:00,205
646,8:46:00,199
644,8:44:00,198
586,7:46:00,198
573,7:33:00,198
588,7:48:00,196
648,8:48:00,192
649,8:49:00,191
582,7:42:00,191
638,8:38:00,190


In [70]:
px.bar(q_3.head(10) , 
x = 'time',
y = 'number_of_trips',
range_y=[0,250] ,
text= 'number_of_trips',
color="time",
title='Top 10 Numbers of trips by time.',
labels={'number_of_trips':''},
template='xgridoff')

In [71]:
qq_3 = American.groupby(['AM_PM','time']).agg(number_of_trips = ('time','count')).sort_values(by='number_of_trips' , ascending=False).reset_index()
qq_3

,AM_PM,time,number_of_trips
0,AM,7:39:00,120
1,AM,8:49:00,119
2,AM,7:46:00,118
3,PM,11:22:00,113
4,AM,7:45:00,113
...,...,...,...
1435,AM,4:33:00,24
1436,AM,5:09:00,23
1437,AM,3:59:00,22
1438,AM,3:38:00,22


In [72]:
px.bar(qq_3, 
x = 'time',
y = 'number_of_trips',
text= 'number_of_trips',
color="AM_PM",
hover_name='time',
title='Numbers of trips by time.',
labels={'number_of_trips':''},
template='seaborn')

__4-What is the most day people asked for trips in it__

In [73]:
q_4 = pd.DataFrame(American.days.value_counts()).reset_index().rename(columns={'index':'day','days':'number_of_trips'})
q_4

,day,number_of_trips
0,Saturday,16552
1,Sunday,14511
2,Friday,14366
3,Tuesday,12011
4,Thursday,11844
5,Wednesday,11373
6,Monday,11055


In [74]:
px.line(q_4 ,
x = 'day' , 
y='number_of_trips' , 
line_shape="spline" ,
color_discrete_sequence=['whitesmoke'],
template='plotly_dark',
markers="markers" ,
hover_name='number_of_trips' ,
labels={'number_of_trips':''},
title="Total number of trips by days.")

__5-What is the number of trips for each day?__

In [75]:
q_5 = American.set_index('complete_date')
q_5 = q_5[['pick_up_address']].resample('d').agg(number_of_trips =('pick_up_address','count')).reset_index()
q_5

,complete_date,number_of_trips
0,2014-07-01,921
1,2014-07-02,1028
2,2014-07-03,1068
3,2014-07-04,1008
4,2014-07-05,1214
...,...,...
87,2014-09-26,1101
88,2014-09-27,1291
89,2014-09-28,1080
90,2014-09-29,772


In [76]:
px.line(q_5 ,
x = 'complete_date' , 
y='number_of_trips' , 
line_shape="spline" ,
template='gridon',
color_discrete_sequence=['black'],
markers="markers" ,
hover_name='number_of_trips' ,
labels={'number_of_trips':''},
title="Total number of trips by days.")

__6-What is the number of trips for each month?__

In [77]:
q_6 = pd.DataFrame(American.months.value_counts()).reset_index().rename(columns={'index':'months','months':'number_of_trips'})
q_6


,months,number_of_trips
0,August,31483
1,July,30348
2,September,29881


In [78]:
fig = px.pie(q_6,
values='number_of_trips',
names='months',
template='ggplot2',
hole=0.5,
title='Total number of trips by months.',
hover_name='number_of_trips')

fig.update_traces(textfont_size = 20 ,pull = [0.1,0,0])

__7-What is the number of trips for each week?__

In [79]:
q_7 = pd.DataFrame(American.Week_name.value_counts()).reset_index().rename(columns={'index':'Weeks','Week_name':'number_of_trips'})
q_7

,Weeks,number_of_trips
0,Second Week,21224
1,Third Week,20971
2,Fifth Week,17766
3,Fourth Week,17344
4,First Week,14407


In [80]:
px.bar(q_7 , 
x = 'number_of_trips',
y = 'Weeks' , 
base='number_of_trips' ,
color= 'Weeks',
template= 'plotly_dark',
title='Total number of trips by each Week.',
labels={'number_of_trips':''},
text='number_of_trips',
pattern_shape='Weeks',
color_discrete_sequence=['whitesmoke', 'violet', 'wheat', 'white', 'turquoise'])

__8-What are the most regions asked for trips?__

In [81]:
q_8 = pd.DataFrame(American.region.value_counts()).reset_index().head(10)
q_8.columns = ['region','number_of_trips']
q_8

,region,number_of_trips
0,Mott Haven,19994
1,South Bronx,11138
2,Cortlandt,5910
3,Brook Ave Mott Haven,3301
4,Cypress Ave Mott Haven,2481
5,Jackson Ave South Bronx,2218
6,Trinity Ave South Bronx,1666
7,Concord Ave South Bronx,1633
8,Saint Anns Ave Mott Haven,1618
9,Willis Ave Mott Haven,1492


In [82]:
px.line(q_8,
x = 'region',
y = 'number_of_trips',
markers='line',
line_shape='hv',
color='region',
template='simple_white' ,
line_dash='region',
hover_name='number_of_trips')

__9-Is there regions asked for trips only AM or  PM?__

In [83]:
q_9 = American.pivot_table(index='region',columns='AM_PM',values= 'date',aggfunc='count' ,margins=True , margins_name='Total').reset_index()

In [84]:
q_9.columns.name = ''
q_9 = q_9.fillna(0)
q_9 = q_9.convert_dtypes()
q_9 = q_9.drop(index=[1661])

In [85]:
q_9 = q_9.sort_values(by='Total',ascending=False).reset_index(drop=True)




In [86]:
q_9

,region,AM,PM,Total
0,Mott Haven,8970,11024,19994
1,South Bronx,5034,6104,11138
2,Cortlandt,2798,3112,5910
3,Brook Ave Mott Haven,1403,1898,3301
4,Cypress Ave Mott Haven,1131,1350,2481
...,...,...,...,...
1656,Mott Haven,1,0,1
1657,Mount Hope Place Bronx,0,1,1
1658,Muliner Ave Parkchester,0,1,1
1659,"N Broadway YONKERS, NY.",0,1,1


In [87]:
px.scatter(q_9,
x = 'AM',
y = 'PM',
color='region',
template='presentation',

)


__10-What is the most period people ask for trips in it?__

In [88]:
q_10 = pd.DataFrame(American.time_period.value_counts()).reset_index()
q_10.columns = ['time_period','number_of_trips']

In [89]:
px.pie(q_10,values='number_of_trips',names='time_period',template='seaborn',title='Total number of trips by time period' , hover_name='number_of_trips')

__11-What is the total number of trips?__

In [90]:
American.date.count()

91712

__12-What is the average of number trips by day, month, week, hour and minute ?__

In [111]:
q_10 = American.groupby(['days','months','hour','minute','AM_PM']).agg(number_of_trips =('date','count')).reset_index()
q_10

,days,months,hour,minute,AM_PM,number_of_trips
0,Friday,August,1,1,AM,4
1,Friday,August,1,1,PM,8
2,Friday,August,1,2,AM,4
3,Friday,August,1,2,PM,1
4,Friday,August,1,3,AM,7
...,...,...,...,...,...,...
27588,Wednesday,September,12,58,AM,2
27589,Wednesday,September,12,58,PM,1
27590,Wednesday,September,12,59,PM,2
27591,Wednesday,September,12,60,AM,1


In [92]:
q_10.minute.nunique()

60

In [112]:
min = q_10.pivot_table(index=['minute'], values='number_of_trips' , aggfunc="mean").reset_index()
min.columns.name = ''
min.number_of_trips = min.number_of_trips.round(2)

In [94]:
px.bar(min, 
x = 'minute',
y = 'number_of_trips',
text= 'number_of_trips',
color="minute",
hover_name='minute',
title='Average of trips by minute.',
labels={'number_of_trips':''},
template='seaborn',
color_continuous_scale='thermal',
range_y=[0,4]
)

In [95]:
hour = q_10.pivot_table(index=['hour'],columns='AM_PM', values='number_of_trips' , aggfunc="mean").reset_index()
hour.columns.name = ''
hour

,hour,AM,PM
0,1,3.244916,2.875214
1,2,2.647348,2.915888
2,3,2.367631,2.992327
3,4,2.318228,3.272109
4,5,2.436893,3.261421
5,6,2.711590,3.288608
6,7,4.197665,3.596315
7,8,4.278005,3.842881
8,9,3.684909,4.041391
9,10,3.041202,4.130507


In [96]:
fig = go.Figure()
fig.add_trace(go.Box(y = hour.AM , 
                     boxpoints='all',
                     fillcolor= 'blue',
                     jitter = 0.5 ,
                     whiskerwidth= 0.2 ,
                     name = 'AM',
              ))
fig.add_trace(go.Box(y = hour.PM , 
                     boxpoints='all',
                     fillcolor= 'red',
                     jitter = 0.5 ,
                     whiskerwidth= 0.2,
                     name = 'PM'))

fig.update_layout(title = 'Average of trips per hour.' ,
                  yaxis = dict(gridcolor = 'rgb(255,255,255)' ,
                               gridwidth = 3) ,
                  paper_bgcolor = 'rgb(243 ,243 ,243)',
                  plot_bgcolor =  'rgb(243 ,243 ,243)')                                                         

In [97]:
qq_10 = q_10.pivot_table(index='days',columns='months', values='number_of_trips' , aggfunc="mean").reset_index()
qq_10.columns.name = ''

In [98]:
qq_10

,days,August,July,September
0,Friday,4.174963,3.209091,3.394939
1,Monday,2.758621,2.759171,3.176606
2,Saturday,4.527738,3.613187,3.840643
3,Sunday,4.044572,3.198813,3.402530
4,Thursday,2.799058,3.409125,2.954726
5,Tuesday,2.682946,3.287547,3.344211
6,Wednesday,2.729515,3.378947,2.749601


In [99]:
px.line(qq_10 , 
x = 'days',
y=['August','July','September'],
line_shape='spline',
markers='markers',
symbol_sequence=[2] ,
color_discrete_sequence=['skyblue', 'tan', 'silver'] ,
template='plotly_dark' ,
)

__13-What is the top 10 address that has asked for trips and the lowest 10?__

In [100]:
q_13_top10 = pd.DataFrame(American.pick_up_address.value_counts().head(10)).reset_index().rename(columns={'index':'address','pick_up_address':'number_of_trips'})
q_13_top10

,address,number_of_trips
0,"400 Brook Ave Mott Haven, BX",1216
1,"331 E 132nd St Mott Haven, BX",888
2,"545 E 145th St Mott Haven, BX",708
3,"436 E 149th St South Bronx, BX",660
4,"169 Cypress Ave Mott Haven, BX",643
5,"105 Willis Ave Mott Haven, BX",641
6,"535 E 142nd St Mott Haven, BX",567
7,"511 E 146th St South Bronx, BX",528
8,"370 Brook Ave Mott Haven, BX",514
9,"780 Saint Anns Ave South Bronx, BX",483


In [101]:
q_13_low10 = pd.DataFrame(American.pick_up_address.value_counts().tail(10)).reset_index().rename(columns={'index':'address','pick_up_address':'number_of_trips'})
q_13_low10

,address,number_of_trips
0,"834 Beck St Foxhurst, BX",1
1,"23 Union Pl YONKERS, NY.",1
2,"Castle Hill Ave Parkchester, BX",1
3,"145 Bruckner Blvd Mott Haven, BX",1
4,"80 Bruckner Blvd Mott Haven, BX",1
5,"536 E 142nd St Mott Haven, BX.",1
6,"1085 University Ave High Bridge, BX",1
7,"carlos se llama con un colchon, 3 bolsas 501...",1
8,que tiempo 295 a la llamada Bronx,1
9,"542 Southern Blvd South Bronx, BX",1


__14-What is the total of number trips by hour and minute ?__

In [102]:
q_14 = q_10.pivot_table(index='hour',columns='AM_PM', values='number_of_trips' , aggfunc="sum" ).reset_index()
q_14.columns.name = ''

In [103]:
px.line(q_14 , 
x = 'hour',
y=['AM','PM'],
line_shape='spline',
markers='markers',
symbol_sequence=[2] ,
color_discrete_sequence=[ 'violet', 'turquoise'] ,
template='plotly_dark' ,

)

In [104]:
q_14

,hour,AM,PM
0,1,3670,3364
1,2,2695,3432
2,3,2209,3510
3,4,2302,3848
4,5,2510,3855
5,6,3018,3897
6,7,5033,4294
7,8,5232,4696
8,9,4444,4882
9,10,3543,4969


__15-How many region we have?__

In [105]:
q_15 = American.region.nunique()

In [114]:
American.to_csv('Uber_data_sets/American.csv')

In [118]:
American = pd.read_csv('H:/CS50/My prjects/Uber_data_sets/American.csv',parse_dates=['date','complete_date'])

In [122]:
American = American.convert_dtypes()

In [123]:
American

,Unnamed: 0,date,time,pick_up_address,AM_PM,region,complete_date,days,months,weeks,hour,minute,time_period,Week_name
0,0,2014-07-01,12:00:00,"874 E 139th St Mott Haven, BX",AM,Mott Haven,2014-07-01 12:00:00,Tuesday,July,1,12,1,Morning,First Week
1,1,2014-07-01,12:01:00,"628 E 141st St Mott Haven, BX",AM,Mott Haven,2014-07-01 12:01:00,Tuesday,July,1,12,2,Morning,First Week
2,2,2014-07-01,12:01:00,"601 E 156th St South Bronx, BX",AM,South Bronx,2014-07-01 12:01:00,Tuesday,July,1,12,2,Morning,First Week
3,3,2014-07-01,12:01:00,"708 E 138th St Mott Haven, BX",AM,Mott Haven,2014-07-01 12:01:00,Tuesday,July,1,12,2,Morning,First Week
4,4,2014-07-01,12:02:00,"700 E 140th St Mott Haven, BX",AM,Mott Haven,2014-07-01 12:02:00,Tuesday,July,1,12,3,Morning,First Week
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91707,91707,2014-09-30,11:47:00,"593 Oak Ter Mott Haven, BX",PM,Oak Ter Mott Haven,2014-09-30 11:47:00,Tuesday,September,1,11,48,Night,Fifth Week
91708,91708,2014-09-30,11:48:00,"645 Westchester Ave South Bronx, BX",PM,Westchester Ave South Bronx,2014-09-30 11:48:00,Tuesday,September,1,11,49,Night,Fifth Week
91709,91709,2014-09-30,11:51:00,"625 Tinton Ave South Bronx, BX",PM,Tinton Ave South Bronx,2014-09-30 11:51:00,Tuesday,September,1,11,52,Night,Fifth Week
91710,91710,2014-09-30,11:54:00,"350 Saint Anns Ave Mott Haven, BX",PM,Saint Anns Ave Mott Haven,2014-09-30 11:54:00,Tuesday,September,1,11,55,Night,Fifth Week
